# CHIPS-CVN Example Notebook

### Get Example ROOT Hit Maps

In [8]:
!mkdir example_data
!wget -P example_data http://www.hep.ucl.ac.uk/~jtingey/beam_nuel_qel_cc.tar
!tar -C example_data -xvf example_data/beam_nuel_qel_cc.tar
!rm example_data/beam_nuel_qel_cc.tar
!wget -P example_data http://www.hep.ucl.ac.uk/~jtingey/beam_numu_qel_cc.tar
!tar -C example_data -xvf example_data/beam_numu_qel_cc.tar
!rm example_data/beam_numu_qel_cc.tar

--2020-01-31 17:21:17--  http://www.hep.ucl.ac.uk/~jtingey/beam_nuel_qel_cc.tar
Resolving www.hep.ucl.ac.uk (www.hep.ucl.ac.uk)... 128.40.4.8
Connecting to www.hep.ucl.ac.uk (www.hep.ucl.ac.uk)|128.40.4.8|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 241418240 (230M) [application/x-tar]
Saving to: ‘example_data/beam_nuel_qel_cc.tar’

beam_nuel_qel_cc.ta 100%[===================>] 230.23M   112MB/s    in 2.1s    

2020-01-31 17:21:19 (112 MB/s) - ‘example_data/beam_nuel_qel_cc.tar’ saved [241418240/241418240]

./beam_nuel_qel_cc/map/chips_1200_sk1pe/
./beam_nuel_qel_cc/map/chips_1200_sk1pe/filtered_001_sim_map.root
./beam_nuel_qel_cc/map/chips_1200_sk1pe/filtered_006_sim_map.root
./beam_nuel_qel_cc/map/chips_1200_sk1pe/filtered_005_sim_map.root
./beam_nuel_qel_cc/map/chips_1200_sk1pe/filtered_000_sim_map.root
./beam_nuel_qel_cc/map/chips_1200_sk1pe/filtered_007_sim_map.root
./beam_nuel_qel_cc/map/chips_1200_sk1pe/filtered_002_sim_map.root
./beam_nuel_qel_cc/ma

### Convert ROOT Hit Maps to tfrecords

In [9]:
import chipscvn.data as data

# Firstly beam_nuel_qel_cc files
nuel_creator = data.DataCreator("example_data/beam_nuel_qel_cc", "chips_1200_sk1pe", 0.1, 10, True)
nuel_creator.preprocess()

# Secondly beam_numu_qel_cc_file
numu_creator = data.DataCreator("example_data/beam_numu_qel_cc", "chips_1200_sk1pe", 0.1, 10, True)
numu_creator.preprocess()

Processing job 0...
Processing job 0...


In [9]:
import chipscvn.utils as utils
import chipscvn.config as config
import chipscvn.data as data
import chipscvn.models as models
import chipscvn.trainers as trainer

utils.gpu_setup()  # Setup the GPU's so they work on all machines
config = config.process_config("config/example_energy_estimation.yml")
data_loader = data.DataLoader(config)
model = utils.get_model(config)
model.summarise()
trainer = utils.get_trainer(config, model, data_loader)

---  4 Physical GPUs, 4 Logical GPUs ---
Model: "single_par_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 64, 64, 64)        1792      
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 62, 62, 64)        36928     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 31, 31, 128)       73856     
_________________________________________________________________
conv2d_27

In [10]:
trainer.train()

Epoch 1/3
246/246 [==============================] - 7s 28ms/step - loss: 1419183.6848 - mae: 743.1565 - mse: 1419184.1250 - val_loss: 213404.5185 - val_mae: 335.7448 - val_mse: 213404.5156
Epoch 2/3
246/246 [==============================] - 6s 25ms/step - loss: 216531.2696 - mae: 332.3323 - mse: 216531.3281 - val_loss: 183197.9237 - val_mae: 298.3668 - val_mse: 183197.9219
Epoch 3/3
246/246 [==============================] - 6s 25ms/step - loss: 172910.4808 - mae: 293.4533 - mse: 172910.5312 - val_loss: 155775.3617 - val_mae: 267.4023 - val_mse: 155775.3594
